# UltraInsight Project Code
## Description


### Setup

In [2]:
#setup
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns


#insert and read file
filename = "../archive (2)/TWO_CENTURIES_OF_UM_RACES.csv"
df = pd.read_csv(filename)
print(df)
df.head()

/var/folders/r6/y72_rzjs01v5mlmhssn_w27h0000gn/T/ipykernel_50317/1146481134.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing


### Data Preprocessing and Cleaning

In [ ]:
# select most recent 10 years
year_cutoff = df['Year of event'].max() - 10
print(len(df))

df = df.dropna()
df = df.drop_duplicates()
print(len(df))

year_inds = df['Year of event'] >= year_cutoff
df = df[year_inds]
print(len(df))

print(df["Event distance/length"])

In [ ]:
#select km and mi races only, not timed ones
distance = df["Event distance/length"]
non_time_km = distance.str.contains('km', regex=False)
non_time_mi = distance.str.endswith('mi')

#setup 2 different dataframes, one with km and one with mi races
df1 = df[non_time_km]
df2 = df[non_time_mi]
print(df1)
print(df2)
print(len(df1))
print(len(df2))

#Take the mi, mile, and miles races and strip them of their ending letters
conversion_kit = df2["Event distance/length"]
print(conversion_kit)
miles = conversion_kit.str.rstrip('miles')
miles = miles.str.rstrip('mile')
miles = miles.str.rstrip('mi')
print(miles)

#manual cleaning up of commas to convert to string floats
miles = miles.str.replace('37,5','37.5')
miles = miles.str.replace('52,5','52.5')
miles = miles.str.replace('52,4','52.4')
miles = miles.str.replace('67,5','67.5')

#convert mi to km
mi_to_km = 1.60934 * miles.astype(float)
print('In kms')
print(mi_to_km)


In [ ]:
#remove km ending from km df
conversion = df1["Event distance/length"]
#conversion = conversion.rename(columns = {"Event distance/length" : "Event distance"})
#print(conversion)
km = conversion

# non_time_km = distance.str.contains('km', regex=False)
good_inds = ~km.str.contains('.Etappen', regex=True) & \
    ~km.str.contains('.Etap', regex=True) & \
    ~km.str.contains(' Sp', regex=False) & \
    ~km.str.contains('.x.', regex=True) & \
    ~km.str.contains('.stag', regex=True)
print(len(good_inds))
print(len(df1))
# km = km[~km.str.contains('.Etappen', regex=True)]
# km = km[~km.str.contains('.Etap', regex=True)]
# km = km[~km.str.contains(' Sp', regex=False)]
# km = km[~km.str.contains('.x.', regex=True)]
# km = km[~km.str.contains('.stag', regex=True)]
km = km[good_inds]

km = km.str.rstrip('kilometers')
km = km.str.rstrip('kilometer')
km = km.str.rstrip('km')
km = km.str.rstrip('km ')

# km to float
km = km.astype(float)
print(km.mean())
print(mi_to_km.mean())
df1 = df1[good_inds]

# replace column with numeric values
# print(len(df1))
df1['Event distance/length'] = km
df2['Event distance/length'] = mi_to_km
print(df1['Event distance/length'].mean())
print(df2['Event distance/length'].mean())

giga_chad_df = pd.concat([df1, df2])
print(giga_chad_df['Event distance/length'].mean())
giga_chad_df = giga_chad_df.rename(columns = {"Event distance/length" : "Event distance"})
df = giga_chad_df
#combine both dfs
# merged_km = [mi_to_km, km]
# # distance = pd.concat(merged_km)
# #distance = distance.rename(columns = {"Event distance/length" : "Event distance"})
# print(distance.mean())

# #reintegrate back into main dataframe
# df["Event distance/length"].update(distance)
#df["Event distance/length"] = df["Event distance/length"].method(distance)
# print(df)
# acceptable_dist_inds = distance.index
# print(acceptable_dist_inds)
# print(distance)
# df = df[acceptable_dist_inds]
# print(df["Event distance/length"])
# print(df.index.max())
# print(acceptable_dist_inds.max())
# df[df.index.isin(acceptable_dist_inds)]


In [ ]:
#rename age category
df = df.rename(columns = {"Athlete age category" : "Athlete age"})

#remove letters from in front of ages (W,MU,M)
rm_age_cat = df["Athlete age"]
print(rm_age_cat)
age = rm_age_cat.str.lstrip('MU')
age = age.str.lstrip('M')
age = age.str.lstrip('W')
age = age.str.lstrip('F')
age = age.str.lstrip('U')
print(age)
age = age.astype(int)

#reintegrate back into main dataframe
df["Athlete age"].update(age)
print(df)
print(df["Athlete age"].mean())


### Data Analysis

In [ ]:


# Convert "Athlete performance" to float
df["Athlete performance"] = df["Athlete performance"].str.replace(':', '.')
df["Athlete performance"] = df["Athlete performance"].str.replace(' ', '0')
df["Athlete performance"] = df["Athlete performance"].str.rstrip('hour')
df["Athlete performance"] = df["Athlete performance"].str.rstrip('hr')
df["Athlete performance"] = df["Athlete performance"].str.rstrip('h')

# Remove entries containing 'd'
df = df[~df["Athlete performance"].str.contains('d', regex=True)]

# Split "Athlete performance" into hours, minutes, and seconds
df["Athlete performance"] = df["Athlete performance"].str.replace('.', '|', regex=False)
df[["Hours", "Minutes", "Seconds"]] = df["Athlete performance"].str.split(pat='|', expand=True)

# Convert time components to float and calculate total time in minutes
df[["Hours", "Minutes", "Seconds"]] = df[["Hours", "Minutes", "Seconds"]].astype(float)
time_in_mins = 60.0 * df["Hours"] + df["Minutes"] + df["Seconds"] / 60.0

# Calculate average pace per race distance
avg_pace = time_in_mins / df["Event distance"] 

df["Pace(min/km)"] = avg_pace


In [ ]:
# Calculate average pace for specific distances
avg_pace_50km = time_in_mins / df[df["Event distance"].between(50, 51)]["Event distance"] 
avg_pace_80km = time_in_mins / df[df["Event distance"].between(80, 81)]["Event distance"] 
avg_pace_100km = time_in_mins / df[df["Event distance"].between(100, 101)]["Event distance"] 
avg_pace_161km = time_in_mins / df[df["Event distance"].between(160, 161)]["Event distance"] 


# Print average paces
print("Average pace for 50 km race:", avg_pace_50km.mean())
print("Average pace for 80 km race:", avg_pace_80km.mean())
print("Average pace for 100 km race:", avg_pace_100km.mean())
print("Average pace for 161 km race:", avg_pace_161km.mean())

#visualizations

# # Store average paces and distances
# distances = [50, 80, 100, 161]
# average_paces = [avg_pace_50km, avg_pace_80km, avg_pace_100km, avg_pace_161km]

# # Create bar plot
# plt.figure(figsize=(10, 6))
# plt.bar(distances, average_paces, color='skyblue')

# # Add title and labels
# plt.title('Average Pace for Different Race Distances')
# plt.xlabel('Race Distance (km)')
# plt.ylabel('Average Pace (min/km)')

# # Show plot
# plt.show()

In [ ]:
#top 20% paces per race distance

# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create an empty DataFrame to store the top 20% of paces for each range
top_20_percent_df = pd.DataFrame()

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the number of rows corresponding to the top 20% of paces
    top_20_percent_rows = int(0.2 * len(filtered_df))
    
    # Sort the filtered DataFrame by pace in descending order
    sorted_df = filtered_df.sort_values(by="Pace(min/km)", ascending=False)
    
    # Take the top 20% of rows from the sorted DataFrame
    top_20_df = sorted_df.head(top_20_percent_rows)
    
    # Append the top 20% of paces to the DataFrame
    top_20_percent_df = pd.concat([top_20_percent_df, top_20_df])

# Print the DataFrame containing the top 20% of paces for each range
print(top_20_percent_df)

# Calculate the average pace for each race distance group
average_pace_per_distance_list = []

for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = top_20_percent_df[(top_20_percent_df["Event distance"] >= range_start) & 
                                     (top_20_percent_df["Event distance"] < range_end)]
    
    # Calculate the average pace for the top 20% of paces in the current distance range
    average_pace = filtered_df["Pace(min/km)"].mean()
    
    # Store the results in a tuple
    result = ((range_start, range_end), average_pace)
    
    # Append the result to the list
    average_pace_per_distance_list.append(result)

# Print the average pace per distance range
for result in average_pace_per_distance_list:
    print("Distance Range:", result[0], "Average Pace:", result[1])




#### Visualization 1: Top 20% of paces per race distance

In [ ]:
#Visualizations top 20% of paces per race distances

# Create lists to store race distances and corresponding average paces
race_distances = []
average_paces = []

# Iterate over each race distance range
for result in average_pace_per_distance_list:
    # Extract race distance range and average pace
    race_distance_range = f"{result[0][0]}-{result[0][1]}"
    average_pace = result[1]
    
    # Append to the lists
    race_distances.append(race_distance_range)
    average_paces.append(average_pace)

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(race_distances, average_paces, color='skyblue')

# Add title and labels
plt.title('Average Pace per Race Distance (Top 20%)')
plt.xlabel('Race Distance (km)')
plt.ylabel('Average Pace (min/km)')

# Add gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()




In [ ]:
#

# Convert "Athlete performance" column to numeric
df['Athlete performance'] = pd.to_numeric(df['Athlete performance'], errors='coerce')

# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create an empty list to store the results
results = []

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Filter the top 20% of finishers by sorting the DataFrame by "Athlete performance"
    top_20_percent_df = filtered_df.nlargest(int(0.2 * len(filtered_df)), 'Athlete performance')
    
    # Group the data by year
    grouped_by_year = top_20_percent_df.groupby('Year of event')
    
    # Calculate the number of finishers in the top 20% for each year
    finishers_count_by_year = grouped_by_year.size()
    
    # Find the year(s) with the maximum number of finishers in the top 20%
    max_finishers_year = finishers_count_by_year[finishers_count_by_year == finishers_count_by_year.max()].index.tolist()
    
    # Append the results to the list
    results.append({'Race Distance': f'{range_start}-{range_end}', 'Years with Most Finishers': max_finishers_year})

# Convert the list of dictionaries into a DataFrame
most_finishers_years_df = pd.DataFrame(results)

# Print the DataFrame
print(most_finishers_years_df)








# Create empty lists to store x and y data
x_values = []
y_values = []

# Iterate over each row in the DataFrame
for index, row in most_finishers_years_df.iterrows():
    race_distance = row['Race Distance']
    years = row['Years with Most Finishers']
    
    # Append race distance to x_values list
    x_values.extend([race_distance] * len(years))
    
    # Append number of years to y_values list
    y_values.extend(years)

# Create the scatter plot
plt.scatter(x_values, y_values)

# Add labels and title
plt.xlabel('Race Distance Range')
plt.ylabel('Year with Most Finishers')
plt.title('Years with Most Finishers in Top 20% by Race Distance Range')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Show plot
plt.grid(True)
plt.show()


In [ ]:
#stdev for outliers of pace per race distance

# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create empty lists to store lower and upper bound ranges
lower_bound_ranges = []
upper_bound_ranges = []

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df1 = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the mean and standard deviation of pace for the filtered DataFrame
    mean_pace = filtered_df1["Pace(min/km)"].mean()
    std_dev_pace = filtered_df1["Pace(min/km)"].std()
    
    # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
    lower_bound = mean_pace - 1.5 * std_dev_pace
    upper_bound = mean_pace + 1.5 * std_dev_pace
    
    # Add the race distance and lower bound range to the lower_bound_ranges list
    lower_bound_ranges.append((range_start, range_end, lower_bound))
    
    # Add the race distance and upper bound range to the upper_bound_ranges list
    upper_bound_ranges.append((range_start, range_end, upper_bound))

# Convert the lists into DataFrames
lower_bounds_df = pd.DataFrame(lower_bound_ranges, columns=["Start", "End", "Lower Bound"])
upper_bounds_df = pd.DataFrame(upper_bound_ranges, columns=["Start", "End", "Upper Bound"])

# Print the DataFrames
print("Lower Bound Ranges:")
print(lower_bounds_df)
print("\nUpper Bound Ranges:")
print(upper_bounds_df)










# # Define the race distance ranges
# race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# # Create empty lists to store outlier information
# lower_bounds = []
# upper_bounds = []

# # Iterate over each race distance range
# for range_start, range_end in race_distance_ranges:
#     # Filter the DataFrame for the specified race distance range
#     filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
#     # Calculate the mean and standard deviation of pace for the filtered DataFrame
#     mean_pace = filtered_df["Pace(min/km)"].mean()
#     std_dev_pace = filtered_df["Pace(min/km)"].std()
    
#     # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
#     lower_bound = mean_pace - 1.5 * std_dev_pace
#     upper_bound = mean_pace + 1.5 * std_dev_pace
    
#     # Add the race distance and lower bound to the lower_bounds list
#     lower_bounds.append({"Event distance": f"{range_start}-{range_end}", "Lower Bound": lower_bound})
    
#     # Add the race distance and upper bound to the upper_bounds list
#     upper_bounds.append({"Event distance": f"{range_start}-{range_end}", "Upper Bound": upper_bound})

# # Convert the lists into DataFrames
# lower_bounds_df = pd.DataFrame(lower_bounds)
# upper_bounds_df = pd.DataFrame(upper_bounds)

# # Print the DataFrames
# print("Lower Bounds:")
# print(lower_bounds_df)
# print("\nUpper Bounds:")
# print(upper_bounds_df)










# # Define the race distance ranges
# race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# # Create an empty list to store outlier information
# outliers_info = []

# # Iterate over each race distance range
# for range_start, range_end in race_distance_ranges:
#     # Filter the DataFrame for the specified race distance range
#     filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
#     # Calculate the mean and standard deviation of pace for the filtered DataFrame
#     mean_pace = filtered_df["Pace(min/km)"].mean()
#     std_dev_pace = filtered_df["Pace(min/km)"].std()
    
#     # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
#     lower_bound = mean_pace - 1.5 * std_dev_pace
#     upper_bound = mean_pace + 1.5 * std_dev_pace
    
#     # Create a range of values for outliers
#     outlier_range = f"{lower_bound:.2f}-{upper_bound:.2f}"
    
#     # Add the race distances and outlier ranges to the outliers_info list
#     outliers_info.append({"Event distance": f"{range_start}-{range_end}", "Outlier Range": outlier_range})

# # Convert the outliers_info list into a DataFrame
# outliers_pace_range_df = pd.DataFrame(outliers_info)

# # Print the outliers_pace_range_df DataFrame
# print(outliers_pace_range_df)





# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create an empty DataFrame to store outliers
outliers_df = pd.DataFrame(columns=df.columns)

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the mean and standard deviation of pace for the filtered DataFrame
    mean_pace = filtered_df["Pace(min/km)"].mean()
    std_dev_pace = filtered_df["Pace(min/km)"].std()
    
    # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
    lower_bound = mean_pace - 1.5 * std_dev_pace
    upper_bound = mean_pace + 1.5 * std_dev_pace
    
    # Identify outliers within the specified range and append them to the outliers DataFrame
    outliers_within_range = filtered_df[(filtered_df["Pace(min/km)"] < lower_bound) | (filtered_df["Pace(min/km)"] > upper_bound)]
    outliers_df = pd.concat([outliers_df, outliers_within_range])

# Print the outliers DataFrame
print(outliers_df)


In [ ]:
#Visualization 3

# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create an empty list to store outlier information
outliers_info = []

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df2 = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the mean and standard deviation of pace for the filtered DataFrame
    mean_pace = filtered_df2["Pace(min/km)"].mean()
    std_dev_pace = filtered_df2["Pace(min/km)"].std()
    
    # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
    lower_bound = mean_pace - 1.5 * std_dev_pace
    upper_bound = mean_pace + 1.5 * std_dev_pace
    
    # Create a range of values for outliers
    outlier_range = f"{lower_bound:.2f}-{upper_bound:.2f}"
    
    # Add the race distances and outlier ranges to the outliers_info list
    outliers_info.append({"Event distance": f"{range_start}-{range_end}", "Outlier Range": outlier_range})

    # Create boxplot
    sns.boxplot(x=filtered_df2["Event distance"], y=filtered_df2["Pace(min/km)"], ax=plt.gca())
    plt.title(f'Race Distance: {range_start}-{range_end} km')
    plt.xlabel('Event Distance (km)')
    plt.ylabel('Pace (min/km)')
    plt.axhline(lower_bound, color='r', linestyle='--', label='Lower Bound')
    plt.axhline(upper_bound, color='g', linestyle='--', label='Upper Bound')
    plt.legend()
    plt.show()

# Convert the outliers_info list into a DataFrame
outliers_pace_range_df = pd.DataFrame(outliers_info)

# Print the outliers_pace_range_df DataFrame
print(outliers_pace_range_df)


#summary

# Define the race distance ranges
race_distance_ranges = [(50.0, 50.001), (80.0, 80.001), (100.0, 100.001), (160.0, 160.001)]

# Create an empty DataFrame to store outliers
outliers_df = pd.DataFrame(columns=df.columns)

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the mean and standard deviation of pace for the filtered DataFrame
    mean_pace = filtered_df["Pace(min/km)"].mean()
    std_dev_pace = filtered_df["Pace(min/km)"].std()
    
    # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
    lower_bound = mean_pace - 1.5 * std_dev_pace
    upper_bound = mean_pace + 1.5 * std_dev_pace
    
    # Identify outliers within the specified range and append them to the outliers DataFrame
    outliers_within_range = filtered_df[(filtered_df["Pace(min/km)"] < lower_bound) | (filtered_df["Pace(min/km)"] > upper_bound)]
    outliers_df = pd.concat([outliers_df, outliers_within_range])

# Create a boxplot comparing race distances and their outliers using Seaborn
plt.figure(figsize=(10, 6))
sns.boxplot(x="Event distance", y="Pace(min/km)", data=outliers_df)
plt.title('Race Distances and Outliers')
plt.xlabel('Race Distance (km)')
plt.ylabel('Pace (min/km)')
plt.xticks(rotation=45)
plt.show()

# # Combine lower and upper bound ranges into one DataFrame
# bounds_df = pd.merge(lower_bounds_df, upper_bounds_df, on=["Start", "End"])

# # Create a boxplot
# plt.figure(figsize=(10, 6))
# plt.boxplot([bounds_df["Lower Bound"], bounds_df["Upper Bound"]], labels=["Lower Bound", "Upper Bound"])
# plt.title('Box and Whisker Plot of Lower and Upper Bound Ranges for Outliers')
# plt.xlabel('Bound Type')
# plt.ylabel('Bound for Outliers (min/km)')
# plt.grid(True)
# plt.show()






# # Plot upper and lower bound ranges
# plt.figure(figsize=(10, 6))

# # Plot lower bound ranges
# for index, row in lower_bounds_df.iterrows():
#     plt.plot([row['Start'], row['End']], [row['Lower Bound'], row['Lower Bound']], color='blue', linestyle='-', label=f"Lower Bound - Race Distance {row['Start']}-{row['End']}")

# # Plot upper bound ranges
# for index, row in upper_bounds_df.iterrows():
#     plt.plot([row['Start'], row['End']], [row['Upper Bound'], row['Upper Bound']], color='red', linestyle='--', label=f"Upper Bound - Race Distance {row['Start']}-{row['End']}")

# plt.title('Upper and Lower Bound Ranges for Outliers per Race Distance')
# plt.xlabel('Race Distance (km)')
# plt.ylabel('Bound for Outliers (min/km)')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
#random dataset of 500 points

# Generate a random sample of 500 data points
random_sample_df = df.sample(n=500, random_state=42)  # Set random_state for reproducibility

# Print the random sample DataFrame
print(random_sample_df)

NameError: name 'df' is not defined